<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2021/examples/blob/main/example-map-reduce/colab/hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install java first
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
#install hadoop. But we clean up first 
!rm -rf hadoop-3.1.2.tar.gz hadoop-3.1.2 /usr/local/hadoop
#download
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.1.2/hadoop-3.1.2.tar.gz
#untar
!tar -xzvf hadoop-3.1.2.tar.gz >/dev/null
!mv hadoop-3.1.2 /usr/local/hadoop


In [ ]:
#get the data file
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!gunzip title.basics.tsv.gz

In [ ]:
#check path and java home
!echo $JAVA_HOME
os.environ["PATH"]+= os.pathsep + "/usr/local/hadoop/bin"
!echo $PATH

In [ ]:
#check hadoop execution and ensure its in path
!/usr/local/hadoop/bin/hadoop

The hadoop process requires a mapper file and reducer file.

In [9]:
#the magic command with two %% will take the code in cell and copy to a file.
%%file mapper.py
#!/usr/bin/python2.7
import sys

def map_function(title):
    fields = title.strip().split('\t')         # Split title to fields using the data delimeter
    primaryTitle = fields[2]                   # Select the required field
    for word in primaryTitle.strip().split():  # Split primary title by words
        yield word.strip('!"*#$').lower(), 1                          # Use a word as a key

if __name__ == "__main__": 
  for line in sys.stdin:
      # Call map_function for each line in the input
      for key, value in map_function(line):
          # Emit key-value pairs using '|' as a delimeter  
          print(key + "|" + str(value))

Overwriting mapper.py


In [10]:
%%file reducer.py
#!/usr/bin/python2.7
import sys

prev_word = None
value_total = 0

if __name__ == "__main__": 
  for line in sys.stdin:          # For ever line in the input from stdin
      line = line.strip()         # Remove trailing characters
      word, value = line.split("|", 1)
      value = int(value)

      # Remember that Hadoop sorts mapper output by key, and the reducer takes these keys sorted
      if prev_word == word:
          value_total += value
      else:
          if prev_word != None:  # Write result to stdout
              print(prev_word + "|" + str(value_total))


          value_total = value
          prev_word = word

  if prev_word == word:  # Don't forget the last key/value pair
      print(prev_word + "|" + str(value_total))

Writing reducer.py


In [11]:
#give execute permission and clean output dir
!chmod +x mapper.py reducer.py
!rm -rf outputdir/

In [13]:
#run hadoop
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.1.2.jar -input title.basics.tsv -output outputdir -mapper mapper.py -reducer reducer.py -file mapper.py -file reducer.py 

2021-03-06 18:07:08,387 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py] [] /tmp/streamjob1253134723867831992.jar tmpDir=null
2021-03-06 18:07:09,047 INFO impl.MetricsConfig: loaded properties from hadoop-metrics2.properties
2021-03-06 18:07:09,141 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-03-06 18:07:09,141 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-03-06 18:07:09,159 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-03-06 18:07:09,318 INFO mapred.FileInputFormat: Total input files to process : 1
2021-03-06 18:07:09,334 INFO mapreduce.JobSubmitter: number of splits:20
2021-03-06 18:07:09,501 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local889401390_0001
2021-03-06 18:07:09,503 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-03-06 18:07:09,821 INFO mapred.LocalDistributedCache

In [14]:
# The output will be created in the outputdir - see the -output flag above
!ls -l outputdir/
!mv outputdir/part-00000 outputdir/hadoop-out.txt

total 16364
-rw-r--r-- 1 root root 16755216 Mar  6 18:09 part-00000
-rw-r--r-- 1 root root        0 Mar  6 18:09 _SUCCESS


In [15]:
#download file
from google.colab import files
files.download('outputdir/hadoop-out.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>